In [ ]:
#https://github.com/ranahaani/GNewsLinks to an external site. 
#The Twitter API to collect and analyze tweets from around the world: https://developer.twitter.com/en/docs/twitter-apiLinks to an external site. 

In [5]:
pip install feedparser

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install gnews

     |████████████████████████████████| 62 kB 959 kB/s eta 0:00:01
     |████████████████████████████████| 81 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 395 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 188 kB 460 kB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=a3f560e6a948d235f354fe7382f08e06c24261989b23a72e3708fa702a2bca99
  Stored in directory: /Users/derek/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=e301e89411ae7bace3863a77eed250b505f81228fb681f483363cfd0d69cd618
  Stored in directory: /Users/derek/Library/Caches/pip/wheels/83/63/2f/117884c3b19d46b64d3d61690333aa80c88dc14050e269c546
Successfully built bs4 sgmllib3k
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests

In [ ]:
pip install spacy

In [ ]:
#France news RSS

In [552]:
pd.set_option('max_columns',None)
pd.set_option('max_colwidth', None)

# Class Wagnerdata:
## Here was the intended class to be used to clean the data

In [632]:
from gnews import GNews
import feedparser
import spacy
import pandas as pd
from spacy.matcher import DependencyMatcher
import dateutil.parser as parser
import requests
import os
import json
 
"""
A class

•	Date (DATE)
•	Approximate time (LOCAL TIME)
•	Activity associated with the Wagner Group (string) A verb such as kills, tweeted: with context, @mentioned
•	Details linking the event to the Wagner Group (can be text or a URL) (url/text)
•	Names of individuals or entities who were involved (where available) (or entity)
•	Location of event (opts: lat,lng;country or twitter available;)

attributes:
- search_term: Search term to use in RSS feed/API searching
- url: url of feed
- raw_exp_loc: Raw Export Location (stores the articles before processing) Export relevant added articles file location
- api_opts: Restrict OOTB news sites
{ 
    "twitter" : {
        "iterations":5
    },
    'gnews':{
        "iterations:5
    },
    'newsapi.org':{
        "iterations":10
    },
    'bing':{
        "iterations":10
    },
    'newsdata.io':{
        "iterations":5
    }
}

- creds: object that maps to api_opts
for each item in api_opts bearer tokens could be passed here in an object format.

Usage:
get_source('url')
To use the twitter api, you must save a bearer token in a file outside this class, in a folder above called twitter_bearer_token.txt
"""

class Wagnerdata:
   
    def __init__(self, search_term, url, export_loc='', api_opts= {'gnews':{},'newsapi.org':{},'bing':{},'newsdata.io':{},'twitter':{'iterations':5}}, creds=[]):
        self.st = search_term
        self.df = []
        #self.export_loc = export_loc
        self.api_opts = api_opts #['gnews','newsapi.org','bing','newsdata.io','twitter']
        self.url = url
        self.output = []
        #self.creds = creds
    
    #Gets lingusitic features into dataframe and sentiment, if desired.             
    def do_prelim_nlp(self, sents, sentiment=False):
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(sents)
        df = pd.DataFrame([])
        if sentiment == True:
            for sentence in sents:
                df['pol'] =  sentence._.polarity
                df['sub'] =  sentence._.subjectivity     
        
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(sents)
        df = []
        
        #date','approx_time','activity','details','name','location']
        #matcher = DependencyMatcher(nlp.vocab)
        ents = [(e.text, e.label_, e.kb_id_) for e in doc.ents]
        #print('ents:')
        #print(ents)
        action = ''

        #for Ents, if GPE, set place:
        for ent in ents:
            print(ent[2])
            if ent[1] == 'GPE':
                place = ent[0]

        #print('first for:')
        for token in doc:
            #if this token is the root verb, then assign it as the action.
            #print(token.dep_)
            if token.dep_ == 'ROOT':
                action = token.text
            #loop through the entities, if the entity is equal to the token, and the token is the subject, make it the subject.
            for ent in ents:
                #print(token.text+'=='+ent[0]+' & '+token.dep_+'== nsubj')
                if token.text == ent[0] and token.dep_ == 'nsubj':

                    #print('ADJUST ============')
                    action = token.text+' '+action
            #print('end first for:---------------')         
            #loop the tokens again to find any additional People, orgs or proper nouns
            for token in doc:    
                for ent in ents:
                    #print(ent)
                    if ent[1] == 'PERSON':
                        noun = ent[0]
                        print(noun)

                    if ent[1] == 'ORG':
                        noun = ent[0]
                        #print(noun)

                    if token.pos_ == 'PROPN':
                        #could be a Russian name
                        print('si_PROPN')
                        #noun = token.text   

                    #if there is a subject of the root and it is an entity...
                    if token.text == ent[0] and token.dep_ == 'nsubj':
                        #print('ADJUST ============')
                        action = token.text+' '+action
                    #print('adding:-----')
                    #print([action,noun])    

                    df.append([action,noun])

                    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
            #print(df)
            df = pd.DataFrame(df, columns = ['activity','entity'])
            df['place'] = place
            df.drop_duplicates()

            #Block of pattern work in development
            # pattern = [
            #     {
            #         "RIGHT_ID": "anchor_wagner",       # unique name
            #         "RIGHT_ATTRS": {"ORTH": "wagner"}  # token pattern for "wagner"
            #     },
            #     {
            #         "LEFT_ID": "anchor_wagner",
            #         "REL_OP": ">",
            #         "RIGHT_ID": "wagner_verb",
            #         "RIGHT_ATTRS": {"DEP": "ROOT"},
            #     }
            # ]
            # matcher.add("WAGNER", [pattern])
            # matches = matcher(doc)

            # print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
            # # Each token_id corresponds to one pattern dict
            # match_id, token_ids = matches[0]
            # for i in range(len(token_ids)):
            #     print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)    
        
        return df
    
    #transforms twitter API data into dataframe
    def transform_call(test):
        fdat = []
        next_token = test['meta']['next_token']
        #for twitem in test['data']:
            #print(twitem['text'])
        authors = []
        created_dte = []
        locations = []
        activities = []
        for i in test['includes']['users']:
            print(i)
            authors.append(i['username'])
            created_dte.append(i['created_at'])
            activities.append('http://twitter.com/'+i['username']+'/status/')

            try:
                i['location']
            except:
                locations.append('UNKWN')
            else:
                locations.append(i['location'])

        i = 0
        for twitem in test['data']:
            dat = []
            #print(twitem)
            try:
                ents = twitem['entities']
            except:
                print('no entities')
            else:
                for ent in twitem['entities']:
                    #entity = ''
        #             print('ent:=======')
        #             print(ent)
                    entity = ''
                    if ent != 'hashtags':
                        it = ''
                        for it in twitem['entities'][ent]:
        #                     print('It:++++++++++++++++')
        #                     print(it)

                            #urls
                            if ent=='urls':
                                if 'expanded_url' in it:    
                                    activity = it['expanded_url']
                                elif 'url' in it:
                                    activity = it['url']
                                rec = [parser.parse(twitem['created_at']).strftime("%Y-%m-%d"),parser.parse(twitem['created_at']).strftime("%H:%M:%S"),'links: '+activity,twitem['id'],authors[i],locations[i]]

                            #mentions
                            if ent=='mentions' :
                                entity = it['username']  
                                rec = [parser.parse(twitem['created_at']).strftime("%Y-%m-%d"),parser.parse(twitem['created_at']).strftime("%H:%M:%S"),ent+': '+entity,twitem['id'],authors[i],locations[i]]

                            #annotations
                            if ent == 'annotations':
         #                       print('annotate')
                                entity = it['normalized_text']
                                #rec = [parser.parse(twitem['created_at']).strftime("%Y-%m-%d"),parser.parse(twitem['created_at']).strftime("%H:%M:%S"),ent+' '+entity,activities[i]+twitem['id'],authors[i],locations[i]]
                                try:
                                    ttype = it['type']
                                    txt = it['normalized_text']
                                    tprob = it['probability']

                                except:
                                    do = 'nothing'
                                    #print('wagner entity exist')

                                else:
                                    if((it['type'] == 'Organization') & (it['normalized_text'] == 'Wagner Group') & (it['probability'] > .5)):
                                        #print('WAGNER==================================================')
                                        do = 'nothing'

                                    if((it['type'] == 'Place') & (it['probability'] > .5)):
                                        #print('PLACE==================================================')
                                        do = 'nothing'

                                rec = [parser.parse(twitem['created_at']).strftime("%Y-%m-%d"),parser.parse( twitem['created_at']).strftime("%H:%M:%S"),ent+': '+entity,twitem['id'],authors[i],locations[i]]

                            dat.append(rec)
        #        print('list of ents!!=========')

            dat.append([parser.parse( created_dte[i]).strftime("%Y-%m-%d"),parser.parse( created_dte[i]).strftime("%H:%M:%S"),'tweeted: '+twitem['text'],twitem['id'],authors[i],locations[i]])
            i += 1
            fdat = fdat + dat
        # print('end of data item=======================') 
        print(fdat)
        pdat = pd.DataFrame(fdat)
        return pdat

    #function for getting tiwtter bearer token    
    def bearer_oauth(r):
        """
        Method required by bearer token authentication.
        """
        tokenf = open('../twitter_bearer_token.txt', 'r')
        token = tokenf.readlines()
        bear_token = token[0]
        r.headers["Authorization"] = f"Bearer {bear_token}"
        r.headers["User-Agent"] = "v2RecentSearchPython"
        return r
    
    #function for connecting to twitter endpoint
    def connect_to_endpoint(self, url, params, token):
        response = requests.get(url, auth=bearer_oauth, params=params)
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response.json()

    #function for calling twitter API
    def maint(self, query_params, search_url):
        json_response = connect_to_endpoint(search_url, query_params, token)
        print(json.dumps(json_response, indent=4, sort_keys=True))
        return json_response
    
    #function for getting the next page
    def paginate_source(self, url):
        test = 'yes'
    
    def process_data():
        test = 'yes'
        
    #Gets the source of the data to pass for cleaning
    #URL
    #OUTPUT: Dataframe
    def get_source(self, url='', feed=True, api_opt=''):
        #is it API?
        #Do we need credentials?
        #is it RSS?
        #does it paginate?
        obj = []
        fdf = []
        #obj = pd.DataFrame([],'columns'==['date','approx_time','activity','details','name','location'])
        
        if feed == True:   
            #testing URL
            #url = "https://vaibhavkumar.hashnode.dev/rss.xml"
            #url = "http://feeds.bbci.co.uk/news/world/rss.xml"
            #https://www.france24.com/en/rss
      
            rss_feed = feedparser.parse(url)
            # rss_feed.feed.title 
            # rss_feed.feed.link
            # len(rss_feed.entries)
            #Debug section
            for i in range(len(rss_feed.entries)):
                #do = 'nothing'
                
                #print(rss_feed.entries[i].link)
                #print(rss_feed.entries[i].author)
                #print(rss_feed.entries[i].published)
                
                if 'wagner' in rss_feed.entries[i].title.lower():
                    #print(rss_feed.entries[i].title)
                    #print('wagner found')
                    #has Wager references add it to the final return dataframe

                    #returns dataframe with activity, entity, place
                    obj = do_prelim_nlp(rss_feed.entries[i].title)
                    #prepends data, activity, entity, place, 
                    #print(obj)
                    try:
                        dte = parser.parse(rss_feed.entries[i].published)
                    except:
                        dte = 'unkown date'

                    try:
                        tme = dte.time()
                    except:
                        tme = 'unknown time'

                    #adds details
                    obj['details'] = rss_feed.entries[i].link
                    obj['date'] = dte
                    obj['time'] = tme
                    #obj = [['date','time','activity','details','entity','place']]

                    # Getting lists of tags and authors.
                    if hasattr(rss_feed.entries[i], 'tags'):
                        tags = [tag.term for tag in rss_feed.entries[i].tags]

                    if hasattr(rss_feed.entries[i], 'authors'):
                        authors= [author.name for author in rss_feed.entries[0].authors]
                else:
                    do = 'nothing'
                    #print("No wagner found!")
                    #print(obj)
                self.ouput = obj

        #not a feed!    
        else:
            #do API login stuff
            print("DOING TWITTER")
            if 'twitter' in api_opt:     
                print('starting twitter')
                print(self.api_opts['twitter'])
                cnt = int(self.api_opts['twitter']['iterations'])
                fdf = []

                search_url = "https://api.twitter.com/2/tweets/search/recent"
                query_params = {'query': '(Wagner Group) -is:retweet',
                                'user.fields': 'username,created_at,location,url',
                                'tweet.fields':'source,entities,created_at',
                                'expansions':  "author_id,entities.mentions.username",
                                'place.fields':'full_name,country,place_type,country_code'
                               }

                for i in range(0,cnt):  
                    if len(fdf) == 0:
                        test = maint(query_params,search_url)
                        df = transform_call(test)
                        fdf = df
                    else:
                        query_params = query_params = {'query': '('+p1.st+') -is:retweet',
                                'user.fields': 'username,created_at,location,url',
                                'tweet.fields':'source,entities,created_at',
                                'expansions':  "author_id,entities.mentions.username",
                                'place.fields':'full_name,country,place_type,country_code',
                                'next_token':test['meta']['next_token']}
                        test = maint(query_params,search_url)
                        df = transform_call(test)
                        fdf = pd.concat([fdf,df])

            elif api_opt=='newsapi.org':
                test = 'yes'
            else:
                test = 'non'

        obj = pd.DataFrame(obj)
        #print(obj.head())
        if len(fdf) > 0:
            obj = pd.concat([obj,fdf])
        obj = obj.drop_duplicates()
        return obj
    
    

p1 = Wagnerdata("Wagner Group",'',export_loc="export_loc")
tst = p1.get_source('',False,'twitter')
#tst.to_csv('twitter_test.txt')
print(tst.head())

test2 = p1.get_source('https://www.france24.com/en/rss')
test2.head()

test3 = p1.get_source('http://rss.cnn.com/rss/cnn_world.rss')
test3.head()

test4 = p1.get_source('http://rss.cnn.com/rss/cnn_topstories.rss')
test4.head()

test5 = p1.get_source('https://www.state.gov/rss-feed/arms-control-and-international-security/feed/')
test4.head()

test6 = p1.get_source('https://www.state.gov/rss-feed/counterterrorism/feed/')
test6.head()

test6 = p1.get_source('https://rss.nytimes.com/services/xml/rss/nyt/Africa.xml')
test6.head()

test7 = p1.get_source('https://rss.nytimes.com/services/xml/rss/nyt/World.xml')
test7.head()

DOING TWITTER
starting twitter
{'iterations': 5}
200
200
200
200
200
            0         1  \
0  2022-12-13  02:25:01   
1  2015-03-10  00:44:04   
2  2022-12-13  02:22:06   
3  2022-12-13  02:22:06   
4  2022-12-13  02:22:06   

                                                                                                                                            2  \
0              links: http://www.cmovietube.com/world/Putin-Won-rsquo-t-Like-This-Ukraine-Is-Trying-to-Kill-the-Wagner-Group-F1076085344.html   
1  tweeted: Putin Won’t Like This: Ukraine Is Trying to Kill the Wagner Group \n\n#Won | #Ukraine | #Trying | #Wagner https://t.co/fQpXFYbsF0   
2                                                                                                                         annotations: Wagner   
3                                                                                                                      mentions: Techonomist_   
4                                          

,activity,entity,place,details,date,time
0,Ukraine Ukraine,Ukraine,Russia,https://www.nytimes.com/2022/12/12/world/europe/wagner-group-ukraine-strike.html,2022-12-13 00:06:47+00:00,00:06:47
5,Ukraine Ukraine,Wagner Group Gathered,Russia,https://www.nytimes.com/2022/12/12/world/europe/wagner-group-ukraine-strike.html,2022-12-13 00:06:47+00:00,00:06:47
12,Ukraine Ukraine,Hotel,Russia,https://www.nytimes.com/2022/12/12/world/europe/wagner-group-ukraine-strike.html,2022-12-13 00:06:47+00:00,00:06:47
18,Ukraine Ukraine,Members,Russia,https://www.nytimes.com/2022/12/12/world/europe/wagner-group-ukraine-strike.html,2022-12-13 00:06:47+00:00,00:06:47
24,Ukraine Ukraine,Russia,Russia,https://www.nytimes.com/2022/12/12/world/europe/wagner-group-ukraine-strike.html,2022-12-13 00:06:47+00:00,00:06:47


# Working with RSS feedparser

In [541]:
#url = "https://vaibhavkumar.hashnode.dev/rss.xml"
#url = "http://feeds.bbci.co.uk/news/world/rss.xml"
#rss_feed = feedparser.parse(url)
# rss_feed.feed.title 
# rss_feed.feed.link
# len(rss_feed.entries)

for i in range(len(rss_feed.entries)):
    print(rss_feed.entries[i].title)
    print(rss_feed.entries[i].link)
    #print(rss_feed.entries[i].author)
    print(rss_feed.entries[i].published)
    if hasattr(rss_feed.entries[i], 'tags'):
        tags = [tag.term for tag in blog_feed.entries[i].tags] 
    
    #authors= [author.name for author in blog_feed.entries[0].authors]
        print(tags)


China zero Covid: Violent protests in Guangzhou put curbs under strain
https://www.bbc.co.uk/news/world-asia-china-63633109?at_medium=RSS&at_campaign=KARANGA
Tue, 15 Nov 2022 09:48:37 GMT
Trump ally Kari Lake loses to Democrat Katie Hobbs in Arizona governor race
https://www.bbc.co.uk/news/world-us-canada-63627007?at_medium=RSS&at_campaign=KARANGA
Tue, 15 Nov 2022 06:37:03 GMT
Ukraine: Zelensky snubs Russia as he addresses 'G19' at G20
https://www.bbc.co.uk/news/world-asia-63632161?at_medium=RSS&at_campaign=KARANGA
Tue, 15 Nov 2022 11:16:13 GMT
Shireen Abu Aqla: Israel won't co-operate with US death probe
https://www.bbc.co.uk/news/world-middle-east-63631629?at_medium=RSS&at_campaign=KARANGA
Tue, 15 Nov 2022 01:24:09 GMT
Ukraine war: Russia 'deliberately' destroyed Kherson infrastructure - Zelensky
https://www.bbc.co.uk/news/world-europe-63621426?at_medium=RSS&at_campaign=KARANGA
Tue, 15 Nov 2022 05:15:24 GMT
Ukraine war: US and Russian spy chiefs meet face-to-face in Turkey
https://ww

In [540]:
rss_feed

{'bozo': False,
 'entries': [{'title': 'China zero Covid: Violent protests in Guangzhou put curbs under strain',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'http://feeds.bbci.co.uk/news/world/rss.xml',
    'value': 'China zero Covid: Violent protests in Guangzhou put curbs under strain'},
   'summary': "Anger boils over in the industrial city of Guangzhou, exposing cracks in Xi Jinping's signature policy.",
   'summary_detail': {'type': 'text/html',
    'language': None,
    'base': 'http://feeds.bbci.co.uk/news/world/rss.xml',
    'value': "Anger boils over in the industrial city of Guangzhou, exposing cracks in Xi Jinping's signature policy."},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://www.bbc.co.uk/news/world-asia-china-63633109?at_medium=RSS&at_campaign=KARANGA'}],
   'link': 'https://www.bbc.co.uk/news/world-asia-china-63633109?at_medium=RSS&at_campaign=KARANGA',
   'id': 'https://www.bbc.co.uk/news/world-asi

## More RSS feed development with Spacy (for archive purposes of project)

In [103]:
sents ='Ukraine war: Wagner chief Prigozhin defends brutal killing video'
#sents = 'Wagner group finds new thing'

In [582]:
#NLP work

import spacy
import pandas as pd
import dateutil.parser as parser

from spacy.matcher import DependencyMatcher


#date','approx_time','activity','details','name','location']
#matcher = DependencyMatcher(nlp.vocab)

def do_nlp(sents):
    df = []
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sents)
    ents = [(e.text, e.label_, e.kb_id_) for e in doc.ents]
    #print('ents:')
    #print(ents)
    action = ''

    #for Ents, if GPE, set place:
    for ent in ents:
        #print(ent[2])
        if ent[1] == 'GPE':
            place = ent[0]

    #print('first for:')
    for token in doc:
        #if this token is the root verb, then assign it as the action.
        #print(token.dep_)
        if token.dep_ == 'ROOT':
            action = token.text
        #loop through the entities, if the entity is equal to the token, and the token is the subject, make it the subject.
        for ent in ents:
            #print(token.text+'=='+ent[0]+' & '+token.dep_+'== nsubj')
            if token.text == ent[0] and token.dep_ == 'nsubj':

                #print('ADJUST ============')
                action = token.text+' '+action
    #print('end first for:---------------')         
    #loop the tokens again to find any additional People, orgs or proper nouns
    for token in doc:    
        for ent in ents:
            #print(ent)
            if ent[1] == 'PERSON':
                noun = ent[0]
                #print(noun)

            if ent[1] == 'ORG':
                noun = ent[0]
                #print(noun)

            if token.pos_ == 'PROPN':
                #could be a Russian name
                #print('si_PROPN')
                noun = token.text   

            #if there is a subject of the root and it is an entity...
            if token.text == ent[0] and token.dep_ == 'nsubj':
                #print('ADJUST ============')
                action = token.text+' '+action
            
            try: noun
            except:
                noun = 'UNKWN'
            #print('adding:-----')
            #print([action,noun])    
            df.append([action,noun])
            

    #df['action'] = action
#     if (token.tag_ == 'VBG' or token.tag_ == 'VBZ') and token.is_alpha == True:
#         action = token.tag_
#         print(action)
#         print(token)
#         print(type(token.tag_))
    
    #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    #print('printing df ===============================')
    #print(df)
    df = pd.DataFrame(df, columns = ['activity','entity'])
    try: place
    except:
        place = 'UNKWN'
    df['place'] = place
    df = df.drop_duplicates()
    return df
    
fdf = []
for entry in rss_feed.entries:
    #print(entry.title)
    sdf = do_nlp(entry.title)
    #print(sdf.count())
    if 'published' in entry:
        sdf['date'] = parser.parse(entry.published ).strftime("%Y-%m-%d")
        sdf['approx_time'] = parser.parse(entry.published ).strftime("%H-%m-%S")
    if 'title' in entry:
        sdf['title'] = entry.title
    if 'link' in entry:
        sdf['details'] = entry.link
    
    if len(fdf) > 0:
        fdf = pd.concat([fdf,sdf])
    else:
        fdf = sdf



In [586]:
dataset = fdf[fdf['title'].str.contains('Wagner')]
dataset.head()

,activity,entity,place,date,approx_time,title,details
0,defends,Ukraine,Ukraine,2022-11-14,15-11-28,Ukraine war: Wagner chief Prigozhin defends brutal killing video,https://www.bbc.co.uk/news/world-europe-63623285?at_medium=RSS&at_campaign=KARANGA
3,defends,Wagner,Ukraine,2022-11-14,15-11-28,Ukraine war: Wagner chief Prigozhin defends brutal killing video,https://www.bbc.co.uk/news/world-europe-63623285?at_medium=RSS&at_campaign=KARANGA
10,defends,Prigozhin,Ukraine,2022-11-14,15-11-28,Ukraine war: Wagner chief Prigozhin defends brutal killing video,https://www.bbc.co.uk/news/world-europe-63623285?at_medium=RSS&at_campaign=KARANGA


In [ ]:
#more NLP work in progress not for submission:

# pattern = [
#     {
#         "RIGHT_ID": "anchor_wagner",       # unique name
#         "RIGHT_ATTRS": {"ORTH": "wagner"}  # token pattern for "wagner"
#     },
#     {
#         "LEFT_ID": "anchor_wagner",
#         "REL_OP": ">",
#         "RIGHT_ID": "wagner_verb",
#         "RIGHT_ATTRS": {"DEP": "ROOT"},
#     }
# ]
# matcher.add("WAGNER", [pattern])
# matches = matcher(doc)

# print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# # Each token_id corresponds to one pattern dict
# match_id, token_ids = matches[0]
# for i in range(len(token_ids)):
#     print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)    
  